In [0]:
# Register control/tiling_params to Unity Catalog
from pyspark.sql import functions as F

# Set Unity Catalog context
spark.sql("USE CATALOG main")
spark.sql("USE SCHEMA demo")

# ===== Configuration =====
CONTROL_PATH = "abfss://aggregated@trimblegeospatialdemo.dfs.core.windows.net/control/tiling_params"
CONTROL_TABLE = "demo_control_tiling_params"  # Unity Catalog table name

print("=== Registering control/tiling_params to Unity Catalog ===")
print(f"Source path: {CONTROL_PATH}")
print(f"Target table: main.demo.{CONTROL_TABLE}")

# Check if data exists at the path
try:
    df = spark.read.format("delta").load(CONTROL_PATH)
    row_count = df.count()
    print(f"\n✅ Data found: {row_count:,} rows")
    
    # Show schema
    print("\nSchema:")
    df.printSchema()
    
    # Show sample data
    print("\nSample data:")
    df.show(10, truncate=False)
    
    # Register as external table in Unity Catalog
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {CONTROL_TABLE}
        USING DELTA
        LOCATION '{CONTROL_PATH}'
        COMMENT 'Control table for tiling parameters'
    """)
    print(f"\n✅ Table registered in Unity Catalog: main.demo.{CONTROL_TABLE}")
    
    # Verify table registration
    print("\n=== Verify table registration ===")
    spark.sql(f"DESCRIBE EXTENDED {CONTROL_TABLE}").show(50, truncate=False)
    
    # Query registered table
    print("\n=== Query registered table ===")
    result = spark.sql(f"SELECT * FROM {CONTROL_TABLE}")
    print(f"Total rows in table: {result.count():,}")
    result.show(10, truncate=False)
    
    print("\n✅ Complete! Control table registered in Unity Catalog.")
    print(f"   - Delta files: {CONTROL_PATH}")
    print(f"   - Unity Catalog table: main.demo.{CONTROL_TABLE}")
    
except Exception as e:
    if "PATH_NOT_FOUND" in str(e) or "does not exist" in str(e):
        print(f"\n❌ Data does NOT exist at path: {CONTROL_PATH}")
        print("Please create the data first before registering the table.")
    else:
        print(f"\n❌ Error: {str(e)}")
        raise